## 6. Agent : 📐 PropositionalLogicAgent (Définitions)

Cet agent est spécialisé dans l'analyse logique formelle (logique propositionnelle) en utilisant la bibliothèque Java Tweety via JPype.

**Rôle :**
*   Traduire des extraits de texte/arguments en un "Belief Set" PL (format Tweety) (`PLAnalyzer.semantic_TextToPLBeliefSet`, puis `StateManager.add_belief_set`).
*   Générer des requêtes logiques pertinentes pour interroger un Belief Set (`PLAnalyzer.semantic_GeneratePLQueries`).
*   Exécuter ces requêtes via la fonction native interfaçant avec Tweety (`PLAnalyzer.execute_pl_query`, puis `StateManager.log_query_result`). **Nécessite une JVM prête.**
*   Interpréter les résultats bruts des requêtes en langage naturel (`PLAnalyzer.semantic_InterpretPLResult`).
*   Répondre aux tâches assignées par le PM (`StateManager.add_answer`).

**Composants Définis Ci-dessous :**
*   `PropositionalLogicPlugin` (Classe V10)
*   Prompts Sémantiques (`prompt_text_to_pl_v8`, `prompt_gen_pl_queries_v8`, `prompt_interpret_pl_v8`) et Fonction Setup (`setup_pl_kernel`)
*   Instructions Système (`PL_AGENT_INSTRUCTIONS` - V10)

**Syntaxe Cruciale :** Le Belief Set et les requêtes doivent respecter la BNF du `PlParser` de Tweety. L'opérateur `>>` est à éviter.

### 🔌 Classe Plugin : PropositionalLogicPlugin

In [ ]:
# %% CELLULE [6.1] - Classe PropositionalLogicPlugin
# (Remplace une partie de l'ancienne cellule 7bbc31e5)

import logging
import jpype
from typing import List, Optional, Any, Dict
import time

logger = logging.getLogger("Orchestration.AgentPL.Defs")
plugin_logger = logging.getLogger("Orchestration.PLAnalyzerPlugin")

# --- Configuration Logger Plugin ---
if not plugin_logger.handlers and not plugin_logger.propagate:
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); plugin_logger.addHandler(handler); plugin_logger.setLevel(logging.INFO)

# --- Plugin Spécifique PLAnalyzer (Refactorisé V9) ---
class PropositionalLogicPlugin:
    """
    Plugin SK pour l'analyse en logique propositionnelle via Tweety/JPype.
    Contient la logique d'interaction Java et expose une fonction native au kernel.
    """
    _logger: logging.Logger
    _jvm_ok: bool
    _PlParser: Optional[Any] = None
    _SatReasoner: Optional[Any] = None
    _PlFormula: Optional[Any] = None
    _parser_instance: Optional[Any] = None
    _reasoner_instance: Optional[Any] = None

    def __init__(self):
        self._logger = plugin_logger
        self._jvm_ok = False
        self._logger.info("Instance PropositionalLogicPlugin créée.")
        self._initialize_jvm_components()

    def _initialize_jvm_components(self):
        """Vérifie la JVM et charge les classes/instances Tweety."""
        global jvm_ready # Vérifier la variable globale définie dans la cellule JPype
        if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
            self._logger.info("JVM détectée comme prête. Chargement classes Tweety...")
            self._jvm_ok = True
        else:
            self._logger.critical("JVM non prête lors init Plugin PL! Fonctions natives échoueront.")
            self._jvm_ok = False
            return

        try:
            self._PlParser = jpype.JClass("org.tweetyproject.logics.pl.parser.PlParser")
            self._SatReasoner = jpype.JClass("org.tweetyproject.logics.pl.reasoner.SatReasoner")
            self._PlFormula = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")
            self._parser_instance = self._PlParser()
            self._reasoner_instance = self._SatReasoner()
            self._logger.info("Classes et instances Java Tweety chargées.")
        except Exception as e:
            self._logger.critical(f"Erreur chargement classes/instances Tweety: {e}", exc_info=True)
            self._jvm_ok = False

    # --- Méthodes Internes ---
    def _internal_parse_formula(self, formula_string: str) -> Optional[Any]:
        # ... (Code _internal_parse_formula inchangé) ...
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse formula: JVM/Parser non prêt ('{formula_string[:60]}...').")
            return None
        try:
            self._logger.debug(f"Parsing formule: '{formula_string}'")
            return self._parser_instance.parseFormula(formula_string)
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing formule '{formula_string}': {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Formule: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing formule '{formula_string}': {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Formule: {e}") from e

    def _internal_parse_belief_set(self, belief_set_string: str) -> Optional[Any]:
        # ... (Code _internal_parse_belief_set inchangé) ...
        if not self._jvm_ok or not self._parser_instance:
            self._logger.error(f"Parse BS: JVM/Parser non prêt (BS: '{belief_set_string[:60]}...').")
            return None
        try:
            belief_set_string_cleaned = belief_set_string.replace("\\\\n", "\\n")
            self._logger.debug(f"Parsing belief set (nettoyé): '{belief_set_string_cleaned[:100]}...'")
            parsed_bs = self._parser_instance.parseBeliefBase(belief_set_string_cleaned)
            self._logger.debug(f" -> BS parsé (type: {type(parsed_bs)}, taille: {parsed_bs.size()}).")
            return parsed_bs
        except jpype.JException as e_java:
            error_msg = f"Erreur Java parsing BS (extrait: '{belief_set_string[:60]}...'): {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Parsing Belief Set: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python parsing BS: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Parsing Belief Set: {e}") from e

    def _internal_execute_query(self, belief_set_obj: Any, formula_obj: Any) -> Optional[bool]:
        # ... (Code _internal_execute_query inchangé) ...
        if not self._jvm_ok or not self._reasoner_instance or not self._PlFormula:
            self._logger.error("Execute query: JVM/Reasoner/Formula non prêt.")
            return None
        try:
            if not isinstance(formula_obj, self._PlFormula):
                 raise TypeError(f"Objet requête n'est pas un PlFormula (type: {type(formula_obj)})")
            self._logger.debug(f"Exécution requête avec raisonneur '{self._reasoner_instance.getClass().getName()}'...")
            result_java_boolean = self._reasoner_instance.query(belief_set_obj, formula_obj)
            self._logger.debug(f" -> Résultat brut Java: {result_java_boolean}")
            if result_java_boolean is None:
                self._logger.warning("Requête Tweety a retourné null (indéterminé?).")
                return None
            else:
                result_python_bool = bool(result_java_boolean)
                self._logger.info(f" -> Résultat requête Python: {result_python_bool}")
                return result_python_bool
        except jpype.JException as e_java:
            error_msg = f"Erreur Java exécution requête: {e_java.getClass().getName()}: {e_java.getMessage()}"
            self._logger.error(error_msg)
            raise RuntimeError(f"Erreur Exécution Requête Tweety: {e_java.getMessage()}") from e_java
        except Exception as e:
            self._logger.error(f"Erreur Python exécution requête: {e}", exc_info=True)
            raise RuntimeError(f"Erreur Python Exécution Requête: {e}") from e

    # --- Méthode Façade ---
    @kernel_function(
        description="Exécute une requête en Logique Propositionnelle (syntaxe Tweety: !,||,=>,<=>,^^) sur un Belief Set fourni. Retourne le résultat (ACCEPTED, REJECTED, Unknown, ou FUNC_ERROR).",
        name="execute_pl_query"
    )
    def execute_pl_query(self, belief_set_content: str, query_string: str) -> str:
        # ... (Code execute_pl_query inchangé) ...
        self._logger.info(f"Appel execute_pl_query: Query='{query_string}' sur BS ('{belief_set_content[:60]}...')")
        if not self._jvm_ok:
            self._logger.error("execute_pl_query: JVM non prête.")
            return "FUNC_ERROR: JVM non prête ou composants Tweety non chargés."
        try:
            belief_set_obj = self._internal_parse_belief_set(belief_set_content)
            if belief_set_obj is None: return "FUNC_ERROR: Échec parsing Belief Set. Vérifiez syntaxe."
            formula_obj = self._internal_parse_formula(query_string)
            if formula_obj is None: return f"FUNC_ERROR: Échec parsing requête '{query_string}'. Vérifiez syntaxe."
            result_bool: Optional[bool] = self._internal_execute_query(belief_set_obj, formula_obj)
            if result_bool is None:
                result_str = f"Tweety Result: Unknown for query '{query_string}'."
                self._logger.warning(f"Requête '{query_string}' -> indéterminé (None).")
            else:
                result_label = "ACCEPTED (True)" if result_bool else "REJECTED (False)"
                result_str = f"Tweety Result: Query '{query_string}' is {result_label}."
                self._logger.info(f" -> Résultat formaté requête '{query_string}': {result_label}")
            return result_str
        except RuntimeError as e_runtime:
             self._logger.error(f"Erreur exécution (parsing/query): {e_runtime}")
             return f"FUNC_ERROR: {e_runtime}"
        except Exception as e_py:
            error_msg = f"Erreur Python inattendue dans execute_pl_query: {e_py}"
            self._logger.error(error_msg, exc_info=True)
            return f"FUNC_ERROR: {error_msg}"


logger.info("Classe PropositionalLogicPlugin (V10) définie.")


### 📜 Prompts Sémantiques et ⚙️ Fonction Setup (PL)

In [ ]:
# %% CELLULE [6.2] - Prompts Sémantiques et Fonction Setup (PL) - CORRIGÉE
# (Remplace une partie de l'ancienne cellule 7bbc31e5 et corrige l'indentation)

import semantic_kernel as sk
from semantic_kernel.functions import kernel_function # Assurez-vous que kernel_function est importé si nécessaire pour le plugin lui-même
import logging

# S'assurer que les dépendances sont là
if 'PropositionalLogicPlugin' not in globals(): raise NameError("Classe PropositionalLogicPlugin non définie.")

logger = logging.getLogger("Orchestration.AgentPL.Setup")

# --- Fonctions Sémantiques PLAnalyzer (Prompts V8 avec BNF) ---
prompt_text_to_pl_v8 = """
[Instructions]
Transformez le texte fourni ($input) en un belief set PL.
Respectez **STRICTEMENT** la BNF Tweety ci-dessous. Retournez les formules une par ligne, séparées par '\\n'.
N'utilisez **PAS** '&&' ou '>>'. Utilisez les opérateurs !, ||, =>, <=>, ^^.
Utilisez des noms de propositions courts et significatifs (ex: `renewable_essential`, `high_cost`).

```bnf
FORMULASET ::== FORMULA ( "\\n" FORMULA )*
FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
             FORMULA "||" FORMULA | FORMULA "=>" FORMULA | FORMULA "<=>" FORMULA |
             FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
PROPOSITION is a sequence of characters excluding |,&,!,(),=,<,> and whitespace.
(Rappel: N'utilisez PAS '>>', utilisez !, ||, =>, <=> si besoin).

[Texte à Analyser]
{{$input}}
+++++
[Belief Set PL (Syntaxe Tweety Valide, une formule par ligne)]
"""

prompt_gen_pl_queries_v8 = """
[Instructions]
Étant donné un texte original ($input) et un belief set PL ($belief_set), générez 2-3 requêtes PL pertinentes pour vérifier la cohérence ou déduire des informations.
Respectez STRICTEMENT la BNF PL Tweety ci-dessous. Retournez les requêtes une par ligne.
N'utilisez PAS '&&' ou '>>'. Utilisez les opérateurs !, ||, =>, <=>, ^^.
Assurez-vous que les propositions utilisées dans les requêtes existent dans le belief set ou sont des combinaisons logiques de celles-ci.
FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
             FORMULA "||" FORMULA | FORMULA "=>" FORMULA | FORMULA "<=>" FORMULA |
             FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
PROPOSITION is a sequence of characters excluding |,&,!,(),=,<,> and whitespace.
(Rappel: N'utilisez PAS '>>').

[Texte Original]
{{$input}}
[Belief Set PL]
{{$belief_set}}
+++++
[Requêtes PL Générées (Syntaxe Tweety Valide, une par ligne)]
"""

prompt_interpret_pl_v8 = """
[Instructions]
Interprétez en langage naturel clair le résultat Tweety formaté ($tweety_result) pour une ou plusieurs requêtes PL ($queries).
Le résultat Tweety pour chaque requête indique si elle est 'ACCEPTED (True)' ou 'REJECTED (False)' ou 'Unknown'.
Basez votre interprétation sur le belief set PL fourni ($belief_set) et le texte original ($input).

Pour chaque requête :

     Rappelez la requête.
     Indiquez si elle est Acceptée, Rejetée ou Inconnue.
     Expliquez pourquoi en vous référant aux formules spécifiques du belief set qui justifient le résultat.
         Si ACCEPTED: Montrez comment la requête découle logiquement des formules du belief set.
         Si REJECTED: Expliquez que la requête N'EST PAS une conséquence logique du belief set (elle n'est pas prouvable), ce qui ne signifie pas forcément qu'elle est fausse dans l'absolu. Mentionnez si elle contredit des formules.
         Si Unknown: Indiquez que le raisonneur n'a pas pu déterminer.

     Reliez l'interprétation au sens du texte original si pertinent.


Générez une interprétation globale concise et facile à comprendre.

[Texte Original]
{{$input}}
[Belief Set PL]
{{$belief_set}}
[Requêtes Testées (une par ligne)]
{{$queries}}
[Résultats Formatés Tweety Agrégés (un résultat par ligne)]
{{$tweety_result}}
+++++
[Interprétation Détaillée en Langage Naturel]
"""
logger.debug("Prompts sémantiques PL (V8) définis.")

# --- Fonction pour configurer le Kernel spécifique à l'agent PL (V10.1 - Correction Indentation) ---
def setup_pl_kernel(kernel: sk.Kernel, llm_service):
    """
    Configure le kernel pour le PropositionalLogicAgent.
    Ajoute une instance du PropositionalLogicPlugin et les fonctions sémantiques.
    """
    plugin_name = "PLAnalyzer" # Doit être DÉFINI ICI
    logger.info(f"Configuration Kernel pour {plugin_name} (V10.1 - Correction Indentation)...")

    # Instanciation du plugin À L'INTÉRIEUR de la fonction
    pl_plugin_instance = PropositionalLogicPlugin()

    # Ajout du plugin au kernel passé en argument
    if plugin_name in kernel.plugins: logger.warning(f"Plugin '{plugin_name}' déjà présent. Remplacement.")
    kernel.add_plugin(pl_plugin_instance, plugin_name=plugin_name)
    logger.debug(f"Instance du plugin '{plugin_name}' ajoutée/mise à jour.")

    # Configuration des settings LLM
    default_settings = None
    if llm_service:
        try: default_settings = kernel.get_prompt_execution_settings_from_service_id(llm_service.service_id); logger.debug(f"Settings LLM récupérés pour {plugin_name}.")
        except Exception as e: logger.warning(f"Impossible de récupérer settings LLM pour {plugin_name}: {e}")

    # Ajout des fonctions sémantiques au kernel
    semantic_functions = [
        ("semantic_TextToPLBeliefSet", prompt_text_to_pl_v8, "Traduit texte en Belief Set PL (syntaxe Tweety ! || => <=> ^^)."),
        ("semantic_GeneratePLQueries", prompt_gen_pl_queries_v8, "Génère requêtes PL pertinentes (syntaxe Tweety ! || => <=> ^^)."),
        ("semantic_InterpretPLResult", prompt_interpret_pl_v8, "Interprète résultat requête PL Tweety formaté.")
    ]
    for func_name, prompt, description in semantic_functions:
        try:
            kernel.add_function(prompt=prompt, plugin_name=plugin_name, function_name=func_name, description=description, prompt_execution_settings=default_settings)
            logger.debug(f"Fonction sémantique {plugin_name}.{func_name} ajoutée/mise à jour.")
        except ValueError as ve: logger.warning(f"Problème ajout/MàJ {plugin_name}.{func_name}: {ve}")

    # Vérification de la fonction native (façade)
    native_facade = "execute_pl_query"
    if plugin_name in kernel.plugins:
        if native_facade not in kernel.plugins[plugin_name]:
            logger.error(f"ERREUR CRITIQUE: Fonction native {plugin_name}.{native_facade} non enregistrée!")
        else:
            logger.debug(f"Fonction native {plugin_name}.{native_facade} trouvée.")
    else:
        logger.error(f"ERREUR CRITIQUE: Plugin {plugin_name} non trouvé après ajout!")

    logger.info(f"Kernel {plugin_name} configuré (V10.1).")

# Fin de la définition de la fonction setup_pl_kernel

### 📜 Instructions Système : PL_AGENT_INSTRUCTIONS

In [ ]:
# %% CELLULE [6.3] - Instructions Système (PL)
# (Remplace une partie de l'ancienne cellule 7bbc31e5)

import logging

logger = logging.getLogger("Orchestration.AgentPL.Instructions")

# --- Instructions Système PL Agent (V10 - Cohérence Noms Fonctions) ---
PL_AGENT_INSTRUCTIONS_V10 = """
Votre Rôle: Spécialiste en logique propositionnelle utilisant Tweety. Vous devez générer et interpréter des formules logiques en respectant **STRICTEMENT** la syntaxe Tweety.

**Syntaxe Tweety PlParser Requise (BNF) :**
```bnf
FORMULASET ::== FORMULA ( "\\n" FORMULA )*
FORMULA ::== PROPOSITION | "(" FORMULA ")" | FORMULA ">>" FORMULA |
             FORMULA "||" FORMULA | FORMULA "=>" FORMULA | FORMULA "<=>" FORMULA |
             FORMULA "^^" FORMULA | "!" FORMULA | "+" | "-"
PROPOSITION is a sequence of characters excluding |,&,!,(),=,<,> and whitespace.
IMPORTANT: N'utilisez PAS l'opérateur >> (cause des erreurs). Utilisez !, ||, =>, <=>, ^^. Formules séparées par \\n dans les Belief Sets. Propositions courtes et sans espaces (ex: renewable_essential). 

Fonctions Outils Disponibles: 

     StateManager.*: Pour lire/écrire dans l'état (get_current_state_snapshot, add_belief_set, log_query_result, add_answer).
     PLAnalyzer.semantic_TextToPLBeliefSet(input: str): Fonction sémantique pour traduire texte en Belief Set PL.
     PLAnalyzer.semantic_GeneratePLQueries(input: str, belief_set: str): Fonction sémantique pour générer des requêtes PL.
     PLAnalyzer.semantic_InterpretPLResult(input: str, belief_set: str, queries: str, tweety_result: str): Fonction sémantique pour interpréter les résultats.
     PLAnalyzer.execute_pl_query(belief_set_content: str, query_string: str): Fonction native pour exécuter une requête PL via Tweety. Retourne le résultat formaté (ACCEPTED/REJECTED/Unknown/FUNC_ERROR). Nécessite une JVM prête.
     

Processus OBLIGATOIRE à chaque tour: 

     CONSULTER L'ÉTAT: Appelez StateManager.get_current_state_snapshot(summarize=True).
     IDENTIFIER VOTRE TÂCHE: Lisez DERNIER message PM (ID tâche, description). Extrayez task_id.
     EXÉCUTER LA TÂCHE:
         Si Tâche = "Traduire ... en Belief Set PL":
        a.  Récupérer le texte source (arguments/texte brut) depuis l'état ou le message du PM.
        b.  Appelez PLAnalyzer.semantic_TextToPLBeliefSet(input=[Texte source]). Validez mentalement la syntaxe de la sortie (Belief Set string) selon la BNF.
        c.  Si la syntaxe semble OK, appelez StateManager.add_belief_set(logic_type="Propositional", content="[Belief Set string généré]"). Notez bs_id. Si l'appel retourne FUNC_ERROR:, signalez l'erreur.
        d.  Préparez réponse texte indiquant succès et bs_id (ou l'erreur).
        e.  Appelez StateManager.add_answer(task_id="[ID reçu]", author_agent="PropositionalLogicAgent", answer_text="...", source_ids=[bs_id si succès]).
         Si Tâche = "Exécuter ... Requêtes PL" (avec belief_set_id):
        a.  Récupérez le belief_set_content correspondant au belief_set_id depuis l'état (StateManager.get_current_state_snapshot(summarize=False) -> belief_sets). Si impossible (ID non trouvé), signalez erreur via add_answer et stoppez.
        b.  Récupérez le raw_text depuis l'état pour le contexte.
        c.  Appelez PLAnalyzer.semantic_GeneratePLQueries(input=raw_text, belief_set=belief_set_content). Validez mentalement la syntaxe des requêtes générées.
        d.  Initialisez formatted_results_list (pour l'interprétation) et log_ids_list. Pour CHAQUE requête q valide générée:
        i.  Appelez PLAnalyzer.execute_pl_query(belief_set_content=belief_set_content, query_string=q).
        ii. Notez le result_str retourné. Ajoutez-le à formatted_results_list. Si result_str commence par FUNC_ERROR:, loggez l'erreur mais continuez si possible avec les autres requêtes.
        iii.Appelez StateManager.log_query_result(belief_set_id=belief_set_id, query=q, raw_result=result_str). Notez le log_id. Ajoutez log_id à log_ids_list.
        e.  Si AU MOINS UNE requête a été tentée: Concaténez tous les result_str dans aggregated_results_str (séparés par newline \\n). Concaténez les requêtes valides testées dans queries_str.
        f.  Appelez PLAnalyzer.semantic_InterpretPLResult(input=raw_text, belief_set=belief_set_content, queries=queries_str, tweety_result=aggregated_results_str). Notez l'interpretation.
        g.  Préparez réponse texte (l'interpretation). Inclure un avertissement si des erreurs (FUNC_ERROR:) ont été rencontrées pendant l'exécution des requêtes.
        h.  Appelez StateManager.add_answer(task_id="[ID reçu]", author_agent="PropositionalLogicAgent", answer_text=interpretation, source_ids=[belief_set_id] + log_ids_list).
         Si Tâche Inconnue/Erreur Préliminaire: Indiquez-le et appelez StateManager.add_answer(task_id=\"[ID reçu]\", ...) avec le message d'erreur.
         
     

Important: Utilisez TOUJOURS task_id reçu pour add_answer. La syntaxe Tweety est STRICTE. Gérez les FUNC_ERROR: retournés par les outils. Vérifiez que la JVM est prête avant d'appeler execute_pl_query (normalement géré par le plugin, mais soyez conscient).
""" 

PL_AGENT_INSTRUCTIONS = PL_AGENT_INSTRUCTIONS_V10 

logger.info("Instructions Système PL_AGENT_INSTRUCTIONS (V10) définies.") 

## 6.1. Test du Plugin PropositionalLogicPlugin (Tweety) - Commenté

Cette cellule, **commentée par défaut**, contient du code pour tester isolément le `PropositionalLogicPlugin`, en particulier son interaction avec Tweety via JPype.

**Objectif du test (s'il était activé) :**
*   Vérifier que la JVM est prête.
*   Instancier le plugin.
*   Exécuter la fonction native `execute_pl_query` avec un belief set et des requêtes prédéfinis.
*   Analyser les résultats (ACCEPTED/REJECTED/Error).

**Prérequis INDISPENSABLES (si activé) :**
1.  La **JVM doit être démarrée correctement** (voir cellule 2). La variable `jvm_ready` doit être `True`.
2.  La cellule de définition du `PropositionalLogicPlugin` doit avoir été exécutée.

**Statut actuel :** Laissé commenté pour se concentrer sur le flux principal. Peut être utile pour déboguer l'interaction JPype/Tweety si l'agent PL rencontre des problèmes.

In [ ]:
# # %% Test du plugin PropositionalLogicPlugin (V6 - Contradiction Explicite)
# import logging
# import jpype # Importer pour vérifier

# test_pl_logger = logging.getLogger("Orchestration.Test.PLPlugin")
# test_pl_logger.setLevel(logging.INFO)
# if not test_pl_logger.handlers:
#     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); test_pl_logger.addHandler(handler)

# test_pl_logger.info("--- Début Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")

# # Vérifier prérequis
# jvm_ok_for_test = False
# if 'jvm_ready' in globals() and jvm_ready and 'jpype' in globals() and jpype.isJVMStarted():
#     test_pl_logger.info("✅ Prérequis JVM OK.")
#     jvm_ok_for_test = True
# else:
#     test_pl_logger.error("❌ Prérequis JVM NON rempli. Le test natif échouera probablement.")
#     print("❌ ERREUR: JVM non prête. Exécutez la cellule de configuration JPype.")

# if 'PropositionalLogicPlugin' not in globals():
#      test_pl_logger.error("❌ Classe PropositionalLogicPlugin non trouvée.")
#      print("❌ ERREUR: Classe PropositionalLogicPlugin non définie.")
#      jvm_ok_for_test = False # Empêcher la suite

# if jvm_ok_for_test:
#     try:
#         # 1. Instanciation
#         test_pl_logger.info("1. Instanciation de PropositionalLogicPlugin...")
#         plugin_instance_pl_test = PropositionalLogicPlugin()
#         test_pl_logger.info("   Instance créée.")

#         # 2. Définition données de test (CORRIGÉ V6: Ajout 'imported_fossils' pour rendre la contradiction claire)
#         test_belief_set = """essential_law
# !high_cost || !false_debate
# false_debate || long_term_cost_inaction
# high_cost
# renewable_creates_jobs
# imported_fossils
# !imported_fossils || dependency
# # La ligne '!renewable_creates_jobs || !dependency' est enlevée car elle contredit directement les faits 5 et 6/7
# irresponsible_ignore
# !essential_law || !long_term_cost_inaction || !renewable_creates_jobs || !dependency || !irresponsible_ignore
# """
#         # Les requêtes restent les mêmes
#         test_query_1 = "essential_law" # Attendu: ACCEPTED
#         test_query_2 = "long_term_cost_inaction" # Attendu: ACCEPTED
#         test_query_3 = "cheap_law" # Attendu: REJECTED/Unknown
#         test_query_4 = "!irresponsible_ignore" # Attendu: ACCEPTED (car KB contradictoire)
#         test_query_5 = "non_existent" # Attendu: REJECTED/Unknown
#         test_query_6 = "essential_law || !essential_law" # Tautologie, Attendu: ACCEPTED
#         test_query_7 = "essential_law && !essential_law" # Contradiction, Attendu: REJECTED (si KB était cohérent), ACCEPTED (si KB contradictoire)

#         print("\n--- Données de Test (Contradiction Explicite) ---")
#         print(f"Belief Set:\n{test_belief_set}")
#         queries_to_test = [test_query_1, test_query_2, test_query_3, test_query_4, test_query_5, test_query_6, test_query_7]
#         for i, q in enumerate(queries_to_test): print(f"Requête {i+1}: {q}")


#         # 3. Exécution des requêtes
#         test_pl_logger.info("\n2. Test: native_ExecutePLQuery sur Belief Set Complet V6...")

#         print("\n--- Résultats Tests Requêtes ---")
#         all_queries_passed = True # Optimiste
#         results_summary = []

#         for i, query in enumerate(queries_to_test):
#             test_pl_logger.info(f"   Exécution requête {i+1}: '{query}'")
#             result = plugin_instance_pl_test.native_ExecutePLQuery(test_belief_set, query)
#             test_pl_logger.info(f"   Résultat brut requête {i+1}: {result}")
#             print(f"Résultat Requête {i+1} ('{query}'): {result}")
#             results_summary.append(result)

#             # Vérification (ATTENTION: si KB contradictoire, tout devrait être ACCEPTED sauf erreur)
#             is_error = "FUNC_ERROR:" in result
#             is_accepted = "ACCEPTED (True)" in result
#             is_rejected = "REJECTED (False)" in result or "Unknown" in result # Grouper REJECTED et Unknown

#             if is_error:
#                 test_pl_logger.error(f"   !!! Erreur détectée pour la requête {i+1} : {result} !!!")
#                 all_queries_passed = False
#             elif i == 2 or i == 4 : # cheap_law (Query 3), non_existent (Query 5) -> Devrait être ACCEPTED si KB contradictoire
#                  if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#                  else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # else: # Toutes les autres requêtes (1, 2, 4, 6, 7) devraient être ACCEPTED si KB contradictoire
#             #      if not is_accepted: all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire)")
#             #      else: test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")
#             # Simplification: on s'attend à ce que tout soit ACCEPTED
#             else:
#                 if not is_accepted:
#                     all_queries_passed = False; test_pl_logger.warning(f"Req {i+1} ('{query}') résultat inattendu (devrait être ACCEPTED car KB contradictoire, obtenu: {'REJECTED/Unknown' if is_rejected else result})")
#                 else:
#                     test_pl_logger.info(f"   Req {i+1} ('{query}') semble OK (ACCEPTED comme attendu pour KB contradictoire).")


#         print("\n--- Vérification Globale Requêtes (Attente: Tout ACCEPTED car KB contradictoire) ---")
#         if all_queries_passed:
#             test_pl_logger.info("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED, comportement attendu pour un KB contradictoire.")
#             print("✅✅✅ Tous les tests de requêtes ont retourné ACCEPTED.")
#         else:
#             test_pl_logger.error("❌❌❌ Le comportement face à la contradiction est inattendu. Certaines requêtes n'ont pas retourné ACCEPTED. Vérifiez les logs et le comportement de Tweety SatReasoner.")
#             print("❌❌❌ Comportement inattendu face à la contradiction. Vérifiez les logs.")
#             print("Résultats obtenus :", results_summary)


#     except Exception as e:
#         test_pl_logger.critical(f"Erreur majeure lors du test du plugin PL: {e}", exc_info=True)
#         print(f"\n !!! ERREUR CRITIQUE PENDANT LE TEST PL : {e} !!!")
# else:
#     print("\n--- Test PL Plugin Sauté (Prérequis non remplis) ---")

# test_pl_logger.info("--- Fin Test PropositionalLogicPlugin (V6 - Contradiction Explicite) ---")